<a href="https://colab.research.google.com/github/masies/CRA/blob/main/Final_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece==0.1.94
!pip install -q torch==1.4.0 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install -q transformers==3.5.0 fast-trees
!pip install --upgrade bleu

!git clone -q https://github.com/microsoft/CodeXGLUE.git

from bleu import file_bleu
import statistics
import os.path


from google.colab import auth
auth.authenticate_user()
project_id = 'helical-loop-303918'
bucket_name = 'code_review_automation'

!gcloud config set project {project_id}
!gsutil -m cp -r gs://{bucket_name}/results_big_model/data .

     |████████████████████████████████| 1.1MB 7.8MB/s 
     |████████████████████████████████| 753.4MB 21kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 1.3MB 7.5MB/s 
     |████████████████████████████████| 1.1MB 30.6MB/s 
     |████████████████████████████████| 901kB 50.0MB/s 
     |████████████████████████████████| 2.9MB 48.6MB/s 
     |████████████████████████████████| 112kB 42.6MB/s 
     |████████████████████████████████| 163kB 44.0MB/s 
     |████████████████████████████████| 71kB 8.3MB/s 
  Created wheel for bleu: filename=bleu-0.3-cp37-none-any.whl size=5802 sha256=583d2d18cc910070df4998237480a5463ad706d7da24c470bf50ba71ae26095f
  Stored in directory: /root/.cache/pip/wheels/a9/95/e7/cb43a1c509c38fedbee6223963e34a51a94d8991f3b3e1888e
  Created wheel for efficiency: f

In [ ]:
def getFileAsList(path):
  with open(path) as f:
    return f.read().splitlines()

class Analizer:
  def __init__(self, target, predictions):
    # print("target -> ", target)
    # print("predictions -> ", predictions)

    self.target = target
    self.predictions = predictions
    self.isPP = False
    self.isAPP = False
    self.codeBLEU = 0
    self.bleu = 0

    for prediction in self.predictions:
      # print()
      # print("prediction -> ", prediction)
      # Check if is a perfect prediction
      if self.target == prediction:
        self.isPP = True

      # Check if is an almost perfect prediction
      if "".join(self.target.split(' ')) == "".join(prediction.split(' ')):
        self.isAPP = True

      # Compute code bleu
      with open("code_bleu_target.txt", "w") as target_cb:
        target_cb.write(self.target + "\n")
      with open("code_bleu_prediction.txt", "w") as prediction_cb:
        prediction_cb.write(prediction + "\n")

      bleu_score = file_bleu("./code_bleu_target.txt", "./code_bleu_prediction.txt")
      self.bleu = bleu_score if bleu_score > self.bleu else self.bleu

      try:
        result = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../code_bleu_target.txt --hyp ./../../../../../code_bleu_prediction.txt --lang java --params 0.25,0.25,0.25,0.25
        code_bleu = float(result[1][result[1].index("CodeBLEU score:  ")+17:])
        
      except: 
        result = !cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && python calc_code_bleu.py --refs ./../../../../../code_bleu_target.txt --hyp ./../../../../../code_bleu_prediction.txt --lang java --params 0.333,0.333,0.333,0
        code_bleu = float(result[2][result[2].index("CodeBLEU score:  ")+17:])

      # compute bleu
      self.codeBLEU = code_bleu if code_bleu > self.codeBLEU else self.codeBLEU
    

def getAnalyzedPredictionList(targets, predictions, beam_size):
  analyzed_predictions = []
  for i in range(len(targets)): 
    if i%1000 == 0:
      print("Computing:", i ," of ",len(targets))
    target = targets[i]
    prediction_list = []
    for j in range(beam_size):
      prediction_list.append(predictions.pop(0))
    
    analyzed_predictions.append(Analizer(target, prediction_list))
  return analyzed_predictions

def logResults(results, task, sector, beam_size):
  count_pp = 0
  count_app = 0
  bleus = []
  code_bleus = []

  for r in results:
    bleus.append(r.bleu)
    code_bleus.append(r.codeBLEU)
    if r.isPP:
      count_pp += 1
    if r.isAPP:
      count_app += 1

  with open("./logs/" + task + "_" + sector + "_" + str(beam_size) + ".log", "w") as f:
    f.write("TASK: " + task + "\t SECTION: " + sector + "\t BEAM: " + str(beam_size) + "\t TOT DATA: " + str(len(targets)) + "\n")
    f.write("PP  :\t" + str(count_pp) + "\t" +  str(count_pp/len(targets)*100) + "\n")
    f.write("APP :\t" + str(count_app) + "\t" +  str(count_app/len(targets)*100) + "\n")
    f.write("Mean CodeBLEU :\t" + str(statistics.mean(code_bleus)) + "\n")
    f.write("Median CodeBLEU :\t" + str(statistics.median(code_bleus)) + "\n")
    f.write("Stdev CodeBLEU :\t" + str(statistics.stdev(code_bleus)) + "\n")
    f.write("Mean Bleu :\t" + str(statistics.mean(bleus)) + "\n")
    f.write("Median Bleu :\t" + str(statistics.median(bleus)) + "\n")
    f.write("Stdev Bleu :\t" + str(statistics.stdev(bleus)) + "\n")


In [ ]:
!mkdir logs
!mkdir ./logs/mixture

sector = "under_100"
# sector = "over_100"

# beam_size = 1
beam_size = 3
# beam_size = 5
# beam_size = 10

task = "code_code"
# task = "codeANDcomment_code"
# task = "code_comment"
# task = "marked_code"
# task = "mixture/code_code"
# task = "mixture/codeANDcomment_code"
# task = "mixture/code_comment"

targets = getFileAsList(os.path.join('/content/data/', sector, task, "test.target"))
predictions = getFileAsList(os.path.join('/content/data/', sector, task, "test_" + str(beam_size) + ".predictions"))
results = getAnalyzedPredictionList(targets, predictions, beam_size)
logResults(results, task, sector, beam_size)
!gsutil -m cp -r ./logs gs://{bucket_name}/results_big_model


mkdir: cannot create directory ‘logs’: File exists
mkdir: cannot create directory ‘./logs/mixture’: File exists
Computing: 0  of  7290
Computing: 1000  of  7290
Computing: 2000  of  7290
Computing: 3000  of  7290
Computing: 4000  of  7290
Computing: 5000  of  7290
Computing: 6000  of  7290
Computing: 7000  of  7290
Copying file://./logs/mixture/code_code_under_100_1.log [Content-Type=application/octet-stream]...
Copying file://./logs/mixture/code_code_under_100_10.log [Content-Type=application/octet-stream]...
Copying file://./logs/mixture/code_comment_under_100_5.log [Content-Type=application/octet-stream]...
Copying file://./logs/mixture/codeANDcomment_code_over_100_5.log [Content-Type=application/octet-stream]...
/ [4/4 files][  895.0 B/  895.0 B] 100% Done                                    
Operation completed over 4 objects/895.0 B.                                      
